In [1]:
import os

import battery_parser as bp

In [2]:
root_dir = r'D:\!Science\Физтех\Циклирования\Эксперименты_метро'

In [23]:
files = bp.list_files(directory=root_dir,
                      filetype='xlsx')  #TODO добавить ключ, по которому можно будет регулировать глубину погружения

In [24]:
patterns = [r'([A-Z]{3})',
            r'[^\\]+?№\s?(\d\d)[^\\]*',
            r'(\d\d)[^\\]*',
            r'[A-Za-z-\s№А-Я]*([0-9]{1,3}).*?(?:\\[^\\]*)?',
            r'[^\\]*-(13[0-9])-.-([0-9]).*-([0-9]{14})']
column_names = ['Chem', 'Tech', 'Step_id', 'Pouch', 'Potentiostat', 'Channel', 'Datetime_id']
pattern = r'\\'.join(patterns)
print(pattern)
parser = bp.importing.Regex_parse()
result = parser(strings=files, pattern=pattern, column_names=column_names)

([A-Z]{3})\\[^\\]+?№\s?(\d\d)[^\\]*\\(\d\d)[^\\]*\\[A-Za-z-\s№А-Я]*([0-9]{1,3}).*?(?:\\[^\\]*)?\\[^\\]*-(13[0-9])-.-([0-9]).*-([0-9]{14})


In [25]:
type_convert = {i: str for i in result.columns}
result = result.astype({'Pouch': int}, copy=True)
result = result.astype(type_convert, copy=True)

result['ID'] = result['Chem'] + result['Pouch']
result.insert(0, 'ID', result.pop('ID'))

thermos = {'1': ['NMC14', 'NMC15', 'LTO7', 'LTO6', 'LFP4', 'LFP5'],
           '2': ['LFP2', 'LFP3', 'NMC6', 'NMC7', 'LTO11', 'LTO12'],
           '3': ['NMC12', 'NMC13', 'LTO13', 'LTO3', 'LFP6', 'LFP7']}
protocol = {'gost45': ['NMC6', 'NMC7', 'LTO11', 'LTO12', 'LFP2', 'LFP3', 'LFP10', 'LFP11'],
            'gost60': ['NMC12', 'NMC13', 'LTO13', 'LTO3', 'LFP6', 'LFP7', 'LFP12', 'LFP13'],
            'Imax45': ['NMC14', 'NMC15', 'LTO7', 'LTO6', 'LFP4', 'LFP5', 'LFP9', 'LFP18'],
            'CCroom': ['NMC10', 'NMC11', 'LTO4', 'LTO5', 'LFP1', 'LFP8']}


In [26]:
if bp.modifications.check_dict_intersection(thermos) or bp.modifications.check_dict_intersection(protocol):
    print('Intersection!')


class Find_key:
    def __init__(self, check_dict):
        self.dictionary = check_dict

    def __call__(self, element):
        result = [key for key, value in self.dictionary.items() if element in value]
        if len(result) == 1:
            return result[0]
        return result


thermos_creator = Find_key(thermos)
technique_creator = Find_key(protocol)

In [27]:
result['Protocol'] = result['ID'].apply(technique_creator)
result.insert(2, 'Protocol', result.pop('Protocol'))
result['Thermos'] = result['ID'].apply(thermos_creator)
result.insert(6, 'Thermos', result.pop('Thermos'))


In [28]:
result.sort_values('Step_id', inplace=True)

In [29]:
result_path = os.path.join(r'D:\!Science\Физтех\Циклирования\Результаты-2', 'File_statistics.csv')
bp.exporting.save_experiment(result, result_path)
